# RAG application using open-source models
Code adapted from https://github.com/svpino/llm

In [1]:
!pip install pypdf
!pip install langchain_community
!pip install langchain[docarray]
!pip  install pydantic==1.10.9
!pip install langchain
!pip install pandas
!pip install chromadb
!pip install -qU langchain-text-splitters
!pip install rank_bm25



: 

In [1]:
import os
import pandas as pd
#from dotenv import load_dotenv

#load_dotenv()

#OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
MODEL = "llama3:8b"
MODEL = "llama3:8b-instruct-q4_0"

REPORTING_YEAR = "2023"
COMPANY_NAME = "Hermès"
COMPANY_DESCRIPTION = ""


# Load the questions file

In [2]:
# Read the excel file as dataframe

df_GRI = pd.read_excel("df_GRI_v2.xlsx")
df_GRI.info()
df_GRI.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  20 non-null     object
 1   Question            20 non-null     object
 2   RSEframe            20 non-null     object
 3   FrameNet            20 non-null     object
 4   Definition          20 non-null     object
 5   CoreFE              20 non-null     object
 6   NonCoreFE           20 non-null     object
 7   MultiQuestion       20 non-null     bool  
 8   QuestionsMultiples  10 non-null     object
 9   AnswerTemplate      20 non-null     object
 10  AnswExamples        20 non-null     object
 11  Relevantinfo        8 non-null      object
 12  UnknownExample      20 non-null     object
 13  Example_1_Kering    3 non-null      object
 14  Example_2_Kering    3 non-null      object
 15  Example_3_Kering    3 non-null      object
 16  Commentaires        3 non-nu

,ID,Question,RSEframe,FrameNet,Definition,CoreFE,NonCoreFE,MultiQuestion,QuestionsMultiples,AnswerTemplate,AnswExamples,Relevantinfo,UnknownExample,Example_1_Kering,Example_2_Kering,Example_3_Kering,Commentaires,Keywords,Reponse,RetrievedContext
0,E9,E9. L'organisation a-t-elle agi pour soutenir ...,E9_Soutenir_Resilience_Action_Climatique,Supporting,A Supporter assists to strengthen the Supporte...,"['Supported', 'Supporter']","['Purpose', 'Means', 'Manner', 'Time', 'Instru...",True,"[""La compagnie a adopté des mesures pour accro...","(""resilience"": boolean)","(""resilience"": TRUE) or (""resilience"": FALSE)",❑ Nous avons adopté des mesures pour accroî...,"(""resilience"": NaN)",Conscient du lien entre la préservation du cli...,"Enfin, le Référent climat rend compte au Conse...",Tout en se concentrant sur trois priorités thé...,Le lien entre l'Organisation et les activités ...,"adaptation climatique', 'résilience climatique'","{""La compagnie a adopté des mesures pour accro...","{""La compagnie a adopté des mesures pour accro..."
1,E10,E10. Veuillez déclarer la consommation d'énerg...,E10_Consume_Renewable_Energy_Proportion,Using_resource,An Agent has access to a finite amount of some...,"['Agent', 'Resource', 'Portion', 'Purpose', 'M...","['Time', 'Place', 'Manner']",False,NaN,"(""proportionRenouvelables"": float64)","(""proportionRenouvelables"": 84.4)",Consommation d'énergies renouvelables en % de ...,"(""proportionRenouvelables"": NaN)",76 % de la consommation d’énergie totale du Gr...,L’électricité reste la principale source d’éne...,"Par ailleurs, la part d’électricité renouvelab...",L'acteur/agent tématisé dans la phrase peut êt...,"consommations', 'électricité renouvelable', 'o...","Explication:\n\nAccording to the document, Ker...","[Document(page_content=""5.2.4.2.1 Mesure des c..."
2,E1,E1. L'entreprise a-t-elle élaboré une politiqu...,E1_Set_Standards_Environment_issues,Commitment,A Speaker makes a commitment to an Addressee t...,"['Speaker', 'Message', 'Topic', 'Medium', 'Add...","['Purpose', 'Time', 'Place', 'Manner', 'Period...",True,"[""Action climatique"", ""Eau"", ""Océans"", ""Forêts...","(""politiqueEnvironmentale"": str)","(""politiqueEnvironnementale"": ""Non, et nous n'...",Required topics:;Action climatique;Eau;Océans;...,"(""politiqueEnvironnementale"": NaN)",Dans le cadre des engagements pris par le Grou...,En coordination avec la Présidente du Comité d...,Kering a structuré ses objectifs selon le cadr...,"C'est quoi une ""politique""? Est-ce que les évé...","action climatique', 'action pour le climat', '...",{'Action climatique': 'Explication = Kering a ...,"{'Action climatique': [Document(page_content=""..."
3,E2,"E2. Au cours de la période de reporting, l'ent...",E2_Engagement_Stakeholders,Collaboration,Partner_1 and Partner_2 or a group of Partners...,"['Partner_1', 'Partner_2', 'Undertaking']","['Duration', 'Time', 'Place', 'Manner', 'Means...",True,"[""Action climatique"", ""Eau"", ""Océans"", ""Forêts...","(""engagementStakeholders"": str)",('engagementStakeholders': 'Aucun engagement ...,"Required topics:, Action climatique, Eau, Océa...","(""engagementStakeholders"": NaN)",NaN,NaN,NaN,NaN,NaN,{'Action climatique': 'Explication = Kering a ...,"{'Action climatique': [Document(page_content=""..."
4,E3,E3. Quel type de mesure l'entreprise a-t-elle ...,E3_Prevention_Impact,Preventing_or_letting,A Potential_hindrance or an Agent (via their a...,"[""Agent"", ""Potential_hindrance"", ""Event""]","['Circumstances', 'Degree', 'Event_description...",True,"[""Action climatique"", ""Eau"", ""Océans"", ""Forêts...","(""preventionImpact"": str)","(""preventionImpact"": ""Formation interne / renf...","Sujets environnementaux :, Action climatique, ...","(""preventionImpact"": NaN)",NaN,NaN,NaN,NaN,NaN,{'Action climatique': 'Explication = Kering a ...,"{'Action climatique': [Document(page_content=""..."


In [3]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

#from langchain_openai.embeddings import OpenAIEmbeddings
#from langchain_openai.chat_models import ChatOpenAI

if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL, show_progress=True, num_gpu=1)

#model.invoke("Tell me a joke")

In [ ]:
#model.invoke("Raconte-moi une blague")

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
#chain.invoke("Tell me a joke")

In [5]:
from langchain.prompts import PromptTemplate

template = """
You are an expert in ecology and environmental issues. Your job is to examinate environmental claims of {COMPANY_NAME} based on their financial and sustainability report for the year {REPORTING_YEAR}. You will always reply in French. Answer the question based on the context below and following the given answer format. Always add to your answer actual quotes of the full sentences you used as a source. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question", COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR)

'\nYou are an expert in ecology and environmental issues. Your job is to examinate environmental claims of Hermès based on their financial and sustainability report for the year 2023. You will always reply in French. Answer the question based on the context below and following the given answer format. Always add to your answer actual quotes of the full sentences you used as a source. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [ ]:
# Example of answer context about biodiversity sites
#context_example = "La compagnie Apple Inc. possède 4 usines de production situées dans des zones de biodiversité. Ces sites couvrent 230 hectares de terres et abritent 13 espèces d'oiseaux en voie de disparition.\"}"
ans_format = "{'sites': int64, 'answer_context': str}"
ans_example = "{'sites': 13, 'answer_context': \"La compagnie X...\""
noAns_example = "{'sites': NaN}"
new_temp = f"{template}\nExpected answer format: {ans_format}" + f"\n\nExample of answer: {ans_example} "
new_temp += f"\n\nIf you do not know the answer to the question, reply '{noAns_example}'"
print(new_temp)


You are an expert in ecology and environmental issues. Your job is to examinate environmental claims of {COMPANY_NAME} based on their financial and sustainability report for the year {REPORTING_YEAR}. You will always reply in French. Answer the question based on the context below and following the given answer format. Always add to your answer actual quotes of the full sentences you used as a source. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}

Expected answer format: {'sites': int64, 'answer_context': str}

Example of answer: {'sites': 13, 'answer_context': "La compagnie X..." 

If you do not know the answer to the question, reply '{'sites': NaN}'


In [ ]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

KeyError: "Input to PromptTemplate is missing variables {'COMPANY_NAME', 'REPORTING_YEAR'}.  Expected: ['COMPANY_NAME', 'REPORTING_YEAR', 'context', 'question'] Received: ['context', 'question']"

# Load the document

In [6]:
from langchain_community.document_loaders import PyPDFLoader
#from langchain_core.text_splitter import RecursiveCharacterTextSplitter

#loader = PyPDFLoader("keringra2023.pdf")
loader = PyPDFLoader("deu-2023-vf_vdef.pdf", extract_images=False)
pages = loader.load_and_split()
pages

#starting_page = 177
#ending_page = 280




[Document(page_content='EXERCICE 2023\nDOCUMENT D’ENREGISTREMENT UNIVERSEL', metadata={'source': 'deu-2023-vf_vdef.pdf', 'page': 0}),
 Document(page_content='SOMMAIRE GÉNÉRAL\nHISTORIQUE  1\nCHIFFRES CLÉS 2\nORGANES DE DIRECTION ET DE CONTRÔLE 5\nORGANIGRAMME SIMPLIFIÉ DU GROUPE AU 31 DÉCEMBRE 2023 6\nRAPPORT DE GESTION DU CONSEIL D’ADMINISTRATION : LE GROUPE 9\nMODÈLE ÉCONOMIQUE DE LVMH 9\nPRÉSENTATION DES ACTIVITÉS, FAITS MARQUANTS ET PERSPECTIVES 13\nCOMMENTAIRES SUR LES RÉSULTATS ET LA SITUATION FINANCIÈRE CONSOLIDÉS 35\nÉTHIQUE ET RESPONSABILITÉ 49\nENVIRONNEMENT ET DURABILITÉ 75\nTALENTS ET ATTRACTIVITÉ 119\nSOLIDARITÉ ET MÉCÉNAT 139\nGESTION DES RISQUES FINANCIERS ET OPÉRATIONNELS, CONTRÔLE INTERNE 147\nRAPPORT DE GESTION DU CONSEIL D’ADMINISTRATION : LA SOCIÉTÉ LVMH 165\nRAPPORT DU CONSEIL D’ADMINISTRATION SUR LE GOUVERNEMENT D’ENTREPRISE 177\nÉTATS FINANCIERS 237\nCOMPTES CONSOLIDÉS  237\nCOMPTES DE LA SOCIÉTÉ LVMH MOËT HENNESSY LOUIS VUITTON 315\nAUTRES INFORMATIONS 347\nRENS

In [7]:
# Extract pages WHERE "page" key of pages' "metadata" are between starting_page and ending_page
#pages_RSE = [page for page in pages if page.metadata.get("page") in range(starting_page+1, ending_page+2)]
pages_RSE = pages

# print first page of pages_RSE
print(pages_RSE[-1])


page_content='Pour toute demande d’information :\nLVMH, 22 avenue Montaigne ‑ 75008 Paris\nTéléphone 01 44 13 22 22\nwww.lvmh.fr et www.lvmh.com' metadata={'source': 'deu-2023-vf_vdef.pdf', 'page': 375}


## Text splitter for consistent chunks

In [8]:
# Create a text splitter from the loaded pages using RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Create text_splitter from RecursiveCharacterTextSplitter with French separators
separators = [".", "!", "?", ":", ";", "\n\n"]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200, separators=separators)
mySplittedDocs = text_splitter.split_documents(pages_RSE)

for doc in mySplittedDocs:
    # Clean line breaks doc.page_content
    doc.page_content = doc.page_content.replace("\n", " ")
    # Remove extra whitespaces
    doc.page_content = doc.page_content.replace("  ", " ")
    
print(mySplittedDocs[88])

from langchain_community.vectorstores import Chroma
vectorstore_splittedDocs = Chroma.from_documents(mySplittedDocs, embedding=embeddings)

page_content='RAPPORT DE GESTION DU CONSEIL D’ADMINISTRATION : LE GROUPE Présentation des activités, faits marquants et perspectives 28 Document d’enregistrement universel 2023Bulgari , fondée en 1884, synonyme de créativité et d’excellence dans le monde entier, est unanimement reconnue comme l’un des acteurs majeurs de son secteur. La célèbre Maison italienne tient une place prééminente dans les domaines de la joaillerie, avec une réputation exceptionnelle pour son savoir‑faire dans la combinaison des pierres de couleur et de l’horlogerie, de même qu’un rang significatif dans celui des parfums et des accessoires. Ses lignes iconiques comprennent Serpenti , B.Zero1 , Diva et Octo. LVMH a acquis Bulgari en 2011. Chaumet , joaillier depuis 1780, maintient un savoir‑faire prestigieux qui se reflète dans toutes ses créations, qu’il s’agisse de pièces de Haute Joaillerie, de collections de bijoux ou de montres. Ses lignes majeures sont Joséphine et Liens. Le groupe LVMH a acquis Chaumet en 

OllamaEmbeddings: 100%|██████████| 115/115 [14:04<00:00,  7.34s/it]


In [9]:
retriever_splitDocs = vectorstore_splittedDocs.as_retriever()
retriever_splitDocs.invoke("zones clés pour la biodiversité")

OllamaEmbeddings: 100%|██████████| 1/1 [00:17<00:00, 17.05s/it]


[Document(page_content='. Taux de participation individuelle des Administrateurs en fonction au 31 décembre 2023  aux séances du Comité d’éthique et du développement durable', metadata={'page': 211, 'source': 'deu-2023-vf_vdef.pdf'}),
 Document(page_content='des mannequins met notamment à leur disposition des bonnes pratiques et conseils prodigués par des nutritionnistes et coaches indépendants. La Charte Santé et Sécurité Signée par le Comité exécutif du Groupe en avril 2021, la Charte Santé et Sécurité encadre une démarche complète sur l’ensemble des opérations du groupe LVMH et vise à développer une culture du « zéro accident ». Le Groupe et ses Maisons s’engagent sur cinq piliers : identifier leurs enjeux par la structuration d’une démarche ; élaborer un plan d’action à revoir périodiquement ; suivre la démarche en soumettant notamment les résultats du taux de fréquence aux comités de direction de chacune des Maisons ; engager chaque collaborateur et collaboratrice dans la démarche

## Retrievers

Base vectorstore retriever

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages_RSE, embedding=embeddings)

Chroma retriever

In [13]:
# Create an alternative vectorstore using Chroma embeddings
from langchain_community.vectorstores import Chroma
vectorstore_chroma = Chroma.from_documents(pages_RSE, embedding=embeddings)


OllamaEmbeddings: 100%|██████████| 4/4 [00:33<00:00,  8.47s/it]


In [86]:
retriever_chroma = vectorstore_chroma.as_retriever()
retriever_chroma.invoke("zones clés pour la biodiversité")

OllamaEmbeddings: 100%|██████████| 1/1 [00:43<00:00, 43.30s/it]


[Document(page_content="dans ses opérations, exposition des sites aux aléas climatiques). Concernant sa chaîne amont, Kering travaille notamment sur les axes suivants\xa0 afin de sécuriser ses approvisionnements à long terme et soutenir les solutions fondées sur la nature, en particulier le développement de l’agriculture régénératrice : •sourcing : mise en œuvre des Kering Standards auprès des fournisseurs en visant 100\xa0% de matières premières alignées avec les Standards en 2025. Ces Standards listent notamment les certifications tierces recommandées dans les chaînes d’approvisionnement du Groupe qui garantissent des pratiques agricoles à faible impact et plus forte résilience, telles que l'agriculture biologique\xa0; •support direct à des programmes de sourcing pour certaines matières premières clés \xa0: Kering est par exemple partenaire du Savory Institute pour promouvoir l’approvisionnement en matières premières venant de système de pâturage régénératifs, en particulier pour le 

## Defininig retrievers


In [ ]:
# Import and use nomic embeddings to feed a Chroma vectorstore
#from langchain_nomic.embeddings import NomicEmbeddings
#from langchain_nomic.embeddings import NomicEmbeddings
#vectorstore_nomic = Chroma.from_documents(mydocs, embedding=NomicEmbeddings(model="nomic-embed-text-v1"))

Ensemble retriever

In [10]:
# Create a bm25 and chroma ensemble retriever with equal weights
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(pages_RSE)
# Set the k parameter to 2 which means that the top 2 documents will be retrieved
bm25_retriever.k = 2

# ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever_chroma], weights=[0.5, 0.5])

#ensemble_retriever.invoke("Superficie (en hectares) des sites possédés, loués ou gérés à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).")


NameError: name 'retriever_chroma' is not defined

In [11]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(pages_RSE)
# Set the k parameter to 2 which means that the top 2 documents will be retrieved
bm25_retriever.k = 2
ensemble_retriever_SplitDocs = EnsembleRetriever(retrievers=[bm25_retriever, retriever_splitDocs], weights=[0.5, 0.5])

#ensemble_retriever_SplitDocs.invoke("Superficie (en hectares) des sites possédés, loués ou gérés à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).")

In [20]:
retriever = vectorstore.as_retriever()
retriever.invoke("zones clés pour la biodiversité")

[Document(page_content='p. 96 — Chiffres clés financiers\np. 99 — Chiffres clés extra-financiers', metadata={'source': 'keringra2023.pdf', 'page': 94}),
 Document(page_content='Chiffres clés', metadata={'source': 'keringra2023.pdf', 'page': 92}),
 Document(page_content='Rapport d’activité — 2023', metadata={'source': 'keringra2023.pdf', 'page': 0}),
 Document(page_content="101EXTRA-FINANCIERS CHIFFRES CLÉS 2023\nAAA\nRécompensant ses actions pour atténuer \nles risques climatiques, lutter contre \nla déforestation via ses approvisionnements \net améliorer la gestion de l’eau, Kering \nse voit attribuer le score AAA par la CDP \nClimate A-List.\n11 ans\nPour la onzième année consécutive, Kering \nse classe parmi les industry leaders \ndu Dow Jones Sustainability Index (DJSI) \nWorld et Europe. Le Groupe obtient le score \nde 84/100 dans le cadre du S&P Corporate \nSustainability Assessment, ce qui le place \nen troisième position dans la catégorie Textile, \nhabillement et produits de l

# (IGNORE) Run the pipeline

---  ------              --------------  ----- 
   * Column              Non-Null Count  Dtype 

 * 0   Indicator             3 non-null      object
 * 1   Keywords4Retriever   3 non-null      object
 * 2   Question             3 non-null      object
 * 3   QuestionType          3 non-null      object
* 4   AnswerTemplate       3 non-null      object
* 5   AnswExamples         3 non-null      object
* 6   UnknownExample       3 non-null      object

In [21]:
from operator import itemgetter
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def model_(question, answer_template, answer_examples, unknown_examples):
    template_base = f"""
    You are an expert in ecology and environmental issues. Your job is to answer the question here below about the company {COMPANY_NAME} based on the context given below for the year {REPORTING_YEAR}, and follow the given "Expected answer format". You will always reply in French. Answer the question. If you can't answer the question, reply {unknown_examples}.

    """ + """Context: {context}

    Question: {question}
    """

    template_base_french = f""""
    Vous êtes un expert en écologie et en problèmes environnementaux. Votre travail est de répondre à la question donnée après sur l'entreprise {COMPANY_NAME} sur la base du contexte donné après datant de l'année {REPORTING_YEAR} en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez {unknown_examples}.
    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \\n\\nDonnees_extraites = {answer_template}\\n\\nAnswer_context=[liste of citations litérales du contexte source que vous avez utilisées pour répondre]" + f"\n\nExemple de bonne réponse: Explication = 'La compagnie X a exploité plus de 100 hectares de terres en Amazonie.' Donnes_extraites = {answer_examples}"
    #template = f"{template_base}\nExpected answer format: {answer_template}" + f"\n\nExample of answer: {answer_examples} "
    #template += f"\n\nIf you do not know the answer to the question, reply '{unknown_examples}'"
    
    #print(template_fr)
    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR)
    print(prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR))
    
    retrieved_docs = ensemble_retriever_SplitDocs.invoke(question)
    context = combine_docs(retrieved_docs)
    #chain = prompt | model | parser
    chain = (
    {
        #"context": itemgetter("question") | retriever,
        #"context": itemgetter("question") | ensemble_retriever | combine_docs,
        "context": itemgetter("context"),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"question": question, "context": context})

    return [result, retrieved_docs]



In [22]:
def add_responses_to_excel(input_df):
    # Charger le fichier Excel
    df = input_df
    
    # Créer une nouvelle colonne pour stocker les réponses
    df['Reponse'] = ''
    
    # Parcourir chaque ligne du fichier Excel
    for index, row in df.iterrows():
        # Récupérer la question de la colonne 'question'
        question = row['Question']
        answer_template = row['AnswerTemplate']
        answer_examples = row['AnswExamples']
        unknown_examples = row['UnknownExample']
        
        print(f"Question: {question}")
        #print(f"Answer Template: {answer_template}")
        #print(f"Answer Examples: {answer_examples}")
        #print(f"Unknown Examples: {unknown_examples}")
        
        # Obtenir la réponse en utilisant la fonction model
        response = model_(question, answer_template, answer_examples, unknown_examples)
        
        # Ajouter la réponse à la nouvelle colonne 'reponse'
        df.at[index, 'Reponse'] = response[0]
        df.at[index, 'RetrievedContext'] = response[1]
    
    # Enregistrer le DataFrame modifié dans un nouveau fichier Excel
    return df

# Create blank df
df_output = add_responses_to_excel(df_GRI)
df_output

Question: Veuillez indiquer le nombre et la superficie (en hectares) des sites possédés, loués ou gérés par l'entreprise dans ou à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).
"
    Vous êtes un expert en écologie et en problèmes environnementaux. Votre travail est de répondre à la question donnée après sur l'entreprise Kering sur la base du contexte donné après datant de l'année 2023 en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez ('sites' : NaN, 'hectares' : NaN).
    
Format de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \n\nDonnees_extraites = ('sites' : int64, 'hectares' : float64)\n\nAnswer_context=[liste of citations litérales du contexte source que vous avez utilisées pour répondre]

Exemple de bonne réponse: Explication = 'La compagnie X a exploité p

OllamaEmbeddings: 100%|██████████| 1/1 [00:28<00:00, 28.18s/it]


ValueError: Must have equal len keys and value when setting with an iterable

In [18]:
# Display dataframe with full text
pd.set_option('display.max_colwidth', None)
df_output

,Indicator,Keywords4Retriever,Question,QuestionType,AnswerTemplate,AnswExamples,UnknownExample,Reponse
0,E15,aires protégées zones clés pour la biodiversité (ZCB),"Veuillez indiquer le nombre et la superficie (en hectares) des sites possédés, loués ou gérés par l'entreprise dans ou à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).",Multi Quantity,"('sites' : int64, 'hectares' : float64)","('sites' : 13, 'hectares' : 23.4)","('sites' : NaN, 'hectares' : NaN)","A question about environmental sustainability and corporate social responsibility!\n\nAccording to the text, Kering has not explicitly disclosed the number and surface area of sites owned, leased, or managed by the company in or near protected areas and/or key biodiversity areas (KBAs). Therefore, I cannot provide a specific answer.\n\nHowever, I can highlight that Kering is committed to environmental sustainability and has set targets to reduce its carbon footprint. The company has also established a Regenerative Fund for Nature, which aims to support the transition towards regenerative agriculture practices, covering over one million hectares by 2025. This effort contributes to climate change mitigation and supports biodiversity conservation.\n\nIf you have any further questions or would like me to analyze other environmental claims made by Kering, please feel free to ask!"
1,E16,aires protégées zones clés pour la biodiversité (ZCB),"Veuillez indiquer le nombre des sites possédés, loués ou gérés par l'entreprise dans ou à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).",Single Quantity,('sites' : int64),('sites' : 13),('sites' : NaN),"A question about environmental sustainability!\n\nAfter analyzing the provided text, I found the relevant information:\n\n""Respecter et protéger l’environnement dans nos opérations et notre chaîne de valeur""\n\nThis section mentions that Kering works on securing its supply chain by implementing standards for sustainable agriculture and supporting programs that promote regenerative farming practices.\n\nUnfortunately, there is no specific mention of the number of sites owned, leased, or managed by the company in protected areas (e.g., national parks) or key biodiversity areas (KBA).\n\nHowever, I can provide some general information on Kering's environmental initiatives:\n\n* They support agriculture regenerative projects to promote sustainable farming practices and reduce carbon emissions.\n* They invest in carbon offsetting projects to compensate for their own emissions and those of their supply chain.\n\nPlease note that this answer does not provide the exact number of sites mentioned in your question. If more information becomes available, I'll be happy to update my response!"
2,E17,aires protégées zones clés pour la biodiversité (ZCB),"Veuillez indiquer la superficie (en hectares) des sites possédés, loués ou gérés par l'entreprise dans ou à proximité d'aires protégées et / ou de zones clés pour la biodiversité (KBA).",Single Quantity,('hectares' : float64),('hectares' : 23.4),('hectares' : NaN),"A question about environmental sustainability!\n\nAfter carefully reviewing the text, I found that Gucci has a program to reforest mining sites in the Amazonian forest (Guyane française) with Solicaz, Amasisa, and Sial. The goal is to reforest the entire area (15 hectares) of these alluvial sites, exceeding the regulation's requirement of 30% reforestation by 2025.\n\nTherefore, I will provide an answer in the format you requested:\n\n('hectares' : 15)\n\nPlease note that this refers specifically to the Amazonian forest site mentioned in the text. If there are other areas or sites owned, leased, or managed by Gucci within or near protected areas and/or Key Biodiversity Areas (KBAs), I would need more information to provide a comprehensive answer."


In [81]:
# Export the dataframe to an excel file
df_output.to_excel("df_GRI_output.xlsx", index=False)

In [ ]:
questions = [
    "Qu'est-ce que la restauration d'écosystèmes ?",
    "Quels sont les avantages des écosystèmes ?",
    "Comment les entreprises peuvent aider aux écosystèmes ?",
    "Qui a publié les principes de développement et de gestion des projets ?",
    "What is the goal of Terramatch?",
]
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)
for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

In [ ]:
chain.batch([{"question": q} for q in questions])

In [ ]:
for s in chain.stream({"question": "Qu'est-ce que ça veut dire PNUE ?"}):
    print(s, end="", flush=True)

In [ ]:
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# loader = WebBaseLoader("https://www.ml.school")
# docs = loader.load()
# documents = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200
# ).split_documents(docs)

# documents

# Import FrameNet from NLTK and search frame "Collaboration" to print its frame elements.

In [137]:
#%pip install nltk
import nltk
from nltk.corpus import framenet as fn
# Import FrameNet from NLTK and search frame "Collaboration" to print its frame elements.
# Load FrameNet
#nltk.download('framenet_v17')
#print(fn.frame_by_name('Replacing'))
# Search for the frame "Collaboration"
#frame = fn.frames('Collaboration')
#print(fn.fes(frame='Collaboration'))
#print(fn.fes('money'))
print(sorted(set(fe.name for fe in fn.fes(frame="Experience_bodily_harm"))))
#print(sorted([(fe.frame.name,fe.name) for fe in fn.fes('Money')]))
# Print the frame elements



['Body_part', 'Containing_event', 'Duration', 'Experiencer', 'Frequency', 'Injuring_entity', 'Iterations', 'Manner', 'Place', 'Severity', 'Time']


In [ ]:
# Testing reading literal eval the Docs in txt file
import ast
import re

# Define the Document class
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# Function to split sentences using Recursive Character Text Splitter
def split_sentences(docs):
    # Split the content of all documents in docs into sentences
    splitted_sentences = []
    for doc in docs:

         # Replace \xa0 with a space
        doc.page_content = doc.page_content.replace('\xa0', ' ')
        
        # Replace double whitespaces by single whitespace
        doc.page_content = re.sub(' +', ' ', doc.page_content.strip())

        # Split the content into sentences using the regex splitter
        #sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)(?=\s[A-Z])|\n\n', doc.page_content)
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)(?=\s[A-Z]|\s\d)|\n\n', doc.page_content)
        
        # Replace break lines by whitespace
        sentences = [sentence.replace("\n", " ") for sentence in sentences]

        # Remove spaces at the beginning of the sentences
        sentences = [sentence.strip() for sentence in sentences]

        # Replace double whitespaces by single whitespace
        sentences = [re.sub(' +', ' ', sentence) for sentence in sentences]

        # Remove spaces at the beginning of the sentences
        sentences = [sentence.strip() for sentence in sentences]
        
        # Add a final period to the sentences that do not end with a period
        sentences = [sentence.strip() + "." if not sentence.endswith(".") else sentence for sentence in sentences]

        # Append the sentences to the list
        splitted_sentences.extend(sentences)
    return splitted_sentences


# Function to read the content of a txt file and return the content as a list of Document objects
def read_docs_from_txt_file(file_path):
    # Read the content of the file
    with open(file_path, "r", encoding='utf-8') as file:
        # Read the content of the file
        content = file.read()
        # Convert the string to a list of Document objects
        docs = eval(content)
        return docs
    
# Test the function with a txt file
docs = read_docs_from_txt_file("output/retrieved_context_1_E10.txt")

# Print the first three sentences of splited docs
test_sentences = split_sentences(docs)

# Store the length of all sentences in a set, then print
sentence_lengths = {len(sentence) for sentence in test_sentences}
print(test_sentences[0])

print(test_sentences)



5.2.4.2.1 Mesure des consommations énergétiques des opérations de Kering et résultats des programmes d’amélioration de l’efficacité énergétique Consommations énergétiques (en MWh) 2023 2022 2021Évolution 2023/2022 Énergies renouvelables 281 875 270 780 235 122 +4 % Électricité achetée d’origine renouvelable 266 680 261 548 229 562 +2 % Électricité d’origine renouvelable produite sur site et auto-consommée 14 531 8 478 4 700 +71 % Biomasse 664 754 860 -12 % Énergies non renouvelables 69 255 78 866 90 686 -12 % Électricité achetée 15 144 15 030 20 063 +1 % Gaz naturel 47 592 56 493 63 403 -16 % Fioul domestique 369 1 026 1 834 -64 % Vapeur, chaleur, froid 6 087 5 315 4 486 +15 % GPL 0 86 72 -100 % Carburant pour transport et manutention sur site 63 309 190 -80 % Autres sources d'énergie 0 607 638 -100 % TOTAL 351 130 349 646 325 808 +0,4 % Consommation d’énergie – Périmètre proforma Total énergie périmètre proforma 288 893 303 218 299 995 -5 % dont électricité 237 620 242 504 232 207 -2 

# Run RAG on multiquestion rows

In [12]:
df_comms = pd.read_excel("df_GRI_v2.xlsx")
df_comms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  20 non-null     object
 1   Question            20 non-null     object
 2   RSEframe            20 non-null     object
 3   FrameNet            20 non-null     object
 4   Definition          20 non-null     object
 5   CoreFE              20 non-null     object
 6   NonCoreFE           20 non-null     object
 7   MultiQuestion       20 non-null     bool  
 8   QuestionsMultiples  10 non-null     object
 9   AnswerTemplate      20 non-null     object
 10  AnswExamples        20 non-null     object
 11  Relevantinfo        8 non-null      object
 12  UnknownExample      20 non-null     object
 13  Example_1_Kering    3 non-null      object
 14  Example_2_Kering    3 non-null      object
 15  Example_3_Kering    3 non-null      object
 16  Commentaires        3 non-nu

In [128]:
import pandas as pd

# Your dictionary
dict_test = {'sites': 13, 'answer_context': 'La compagnie X...'}

# Convert the dictionary to a string
#dict_str = str(dict_test)

# Create a DataFrame with the string
df_test = pd.DataFrame([str(dict_test)], columns=['dict'])
df_test.info()
# Print the DataFrame
df_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dict    1 non-null      object
dtypes: object(1)
memory usage: 136.0+ bytes


,dict
0,"{'sites': 13, 'answer_context': 'La compagnie X...'}"


In [129]:
from operator import itemgetter
import ast

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def model_(question, answer_template, answer_examples, unknown_examples):
    template_base = f"""
    You are an expert in ecology and environmental issues. Your job is to answer the question here below about the company {COMPANY_NAME} based on the context given below for the year {REPORTING_YEAR}, and follow the given "Expected answer format". You will always reply in French. Answer the question. If you can't answer the question, reply {unknown_examples}.

    """ + """Context: {context}

    Question: {question}
    """

    template_base_french = f""""
    Vous êtes un expert en écologie et environnement de l'entreprise {COMPANY_NAME}. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année {REPORTING_YEAR} en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez {unknown_examples}.
    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \\n\\nDonnees_extraites = {answer_template}\\n\\nAnswer_context=[citations littérales de la source que vous avez utilisées pour répondre, citation2, citation3]" + f"\n\nExemple de bonne réponse: Explication = 'L'entreprise X a exploité plus de 100 hectares de terres en Amazonie.' Donnes_extraites = {answer_examples}"
    #template = f"{template_base}\nExpected answer format: {answer_template}" + f"\n\nExample of answer: {answer_examples} "
    #template += f"\n\nIf you do not know the answer to the question, reply '{unknown_examples}'"
    
    #print(template_fr)
    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR)
    print(prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR))
    
    retrieved_docs = ensemble_retriever.invoke(question)
    context = combine_docs(retrieved_docs)
    #chain = prompt | model | parser
    chain = (
    {
        #"context": itemgetter("question") | retriever,
        #"context": itemgetter("question") | ensemble_retriever | combine_docs,
        "context": itemgetter("context"),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"question": question, "context": context})
    print(result)
    return [result, retrieved_docs]



def add_responses_to_excel(input_df):
    # Create a copy of input_df
    df = input_df.copy()
    
    # Créer une nouvelle colonne pour stocker les réponses
    df['Reponse'] = ''
    df['RetrievedContext'] = ''
    
    # Parcourir chaque ligne du fichier Excel
    for index, row in df.iterrows():

        # Check if it has multiple questions
        if row['MultiQuestion'] == 1:
            # Read the value on QuestionsMultiples on the first row as a list and store it in a variable
            multiquestions = row['QuestionsMultiples']
            # Convert the string to a list
            multiquestions = ast.literal_eval(multiquestions)

            # Empty dictionary to store the responses
            multiresponses = {}
            multiretrieved_docs = {}

            for q_complement in multiquestions:
                # Fusionner la question complémentaire avec la question principale
                question = row['Question'] + " " + q_complement
                # Obtenir la réponse en utilisant la fonction model
                r_complement = model_(question, row['AnswerTemplate'], row['AnswExamples'], row['UnknownExample'])
                # Stocker question: réponse dans le dictionnaire
                multiresponses[q_complement] = r_complement[0]
                # Stocker question: retrieved_docs dans le dictionnaire
                multiretrieved_docs[q_complement] = r_complement[1]

                # Export r_complement to a txt file
                with open(f"reponse_{index}_{row['ID']}_{q_complement}.txt", "w", encoding='utf-8') as file:
                    file.write(str(r_complement[0]))

                # Export retrieved_docs to a txt file
                with open(f"retrieved_context_{index}_{row['ID']}_{q_complement}.txt", "w", encoding='utf-8') as file:
                    file.write(str(r_complement[1]))

            # Ajouter le dictionnaire multiresponses à la nouvelle colonne 'reponse'
            df.at[index, 'Reponse'] = str(multiresponses)
            df.at[index, 'RetrievedContext'] = str(multiretrieved_docs)

            # Export multiretrieved_docs to a txt file
            with open(f"retrieved_context_{index}_{row['ID']}.txt", "w",  encoding='utf-8') as file:
                file.write(str(multiretrieved_docs))

            # Export multiresponses to a txt file with UTF-8 encoding
            with open(f"reponses_{index}_{row['ID']}.txt", "w", encoding='utf-8') as file:
                file.write(str(multiresponses))

        else:
            # Récupérer la question de la colonne 'question'
            response = model_(row['Question'], row['AnswerTemplate'], row['AnswExamples'], row['UnknownExample'])

            # Ajouter la réponse à la nouvelle colonne 'reponse'
            df.at[index, 'Reponse'] = response[0]
            df.at[index, 'RetrievedContext'] = response[1]
    
    # Enregistrer le DataFrame modifié dans un nouveau fichier Excel
    return df

# Create blank df
df_output = add_responses_to_excel(df_comms)
df_output.info()

I'm ready to answer your question!

Question: (Please provide the question)

Contexte: Kering, 2023

Format attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \n\ln Donnees_extraites = ("resilience": boolean)\n\ Answer_context=[citations littéraires de la source que vous avez utilisées pour répondre, citation2, citation3]

Please provide the question!
Je suis prêt à répondre à votre question !

Veuillez donner la question et le contexte, je vais vous fournir une réponse dans le format attendu.

(Note : comme expert en écologie et environnement de l'entreprise Kering, j'ai accès aux documents internes de l'entreprise, mais si la question ne concerne pas spécifiquement l'entreprise Kering, je pourrai encore fournir une réponse basée sur des informations générales et fiables. Si c'est le cas, je vais signaler cela dans ma réponse.)
I'm an ecologist and environmental expert at Kering. I'll answer the question given below based on the context provide

In [130]:
# Export df_output to an excel file
df_output.to_excel("df_comms_output2.xlsx", index=False)

In [13]:
# Corrected with context and question

from operator import itemgetter
import ast
#install tqdm in pip
#%pip install tqdm
from tqdm import tqdm

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def model_(question, answer_template, answer_examples, unknown_examples):
    

    template_base_french = f""""Vous êtes un expert en écologie et environnement de l'entreprise {COMPANY_NAME}. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année {REPORTING_YEAR} en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez {unknown_examples}.
    """ + """
    Context: {context}

    Question: {question}
    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \\n\\nDonnees_extraites = {answer_template}\\n\\nAnswer_context=[citations littérales de la source que vous avez utilisées pour répondre, citation2, citation3]" + f"\n\nExemple de bonne réponse: Explication = 'L'entreprise X a exploité plus de 100 hectares de terres en Amazonie.' Donnes_extraites = {answer_examples}"
    
    
    #print(template_fr)
    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR)
    print(prompt.format(context="Here is the context", question=question, COMPANY_NAME=COMPANY_NAME, REPORTING_YEAR=REPORTING_YEAR))
    
    retrieved_docs = ensemble_retriever_SplitDocs.invoke(question)
    context = combine_docs(retrieved_docs)
    #chain = prompt | model | parser
    chain = (
    {
        #"context": itemgetter("question") | retriever,
        #"context": itemgetter("question") | ensemble_retriever | combine_docs,
        "context": itemgetter("context"),
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"question": question, "context": context})
    print(result)
    return [result, retrieved_docs]



def add_responses_to_excel(input_df):
    # Create a copy of input_df
    df = input_df.copy()
    
    # Créer une nouvelle colonne pour stocker les réponses
    df['Reponse'] = ''
    df['RetrievedContext'] = ''
    
    # Parcourir chaque ligne du fichier Excel
    for index, row in tqdm(df.iterrows(), desc="Processing Excel rows"):
        rowID = str(row['ID'])
        # Check if it has multiple questions
        if row['MultiQuestion'] == 1:
            # Read the value on QuestionsMultiples on the first row as a list and store it in a variable
            multiquestions = row['QuestionsMultiples']
            # Convert the string to a list
            multiquestions = ast.literal_eval(multiquestions)

            # Empty dictionary to store the responses
            multiresponses = {}
            multiretrieved_docs = {}
            q_complement_index = 0

            for q_complement in tqdm(multiquestions, desc="Processing multiple questions"):
                # Counter of iterations
                q_complement_index += 1

                # Fusionner la question complémentaire avec la question principale
                question = row['Question'] + " " + q_complement
                # Obtenir la réponse en utilisant la fonction model
                r_complement = model_(question, row['AnswerTemplate'], row['AnswExamples'], row['UnknownExample'])
                # Stocker question: réponse dans le dictionnaire
                multiresponses[q_complement] = r_complement[0]
                # Stocker question: retrieved_docs dans le dictionnaire
                multiretrieved_docs[q_complement] = r_complement[1]

                # Export r_complement to a txt file
                with open(f"reponse_{index}_{rowID}_{q_complement_index}.txt", "w", encoding='utf-8') as file:
                    file.write(str(r_complement[0]))

                # Export retrieved_docs to a txt file
                with open(f"retrieved_context_{index}_{rowID}_{q_complement_index}.txt", "w", encoding='utf-8') as file:
                    file.write(str(r_complement[1]))

            # Ajouter le dictionnaire multiresponses à la nouvelle colonne 'reponse'
            df.at[index, 'Reponse'] = str(multiresponses)
            df.at[index, 'RetrievedContext'] = str(multiretrieved_docs)

            # Export multiretrieved_docs to a txt file
            with open(f"retrieved_context_{index}_{rowID}.txt", "w",  encoding='utf-8') as file:
                file.write(str(multiretrieved_docs))

            # Export multiresponses to a txt file with UTF-8 encoding
            with open(f"reponses_{index}_{rowID}.txt", "w", encoding='utf-8') as file:
                file.write(str(multiresponses))

        else:
            # Récupérer la question de la colonne 'question'
            response = model_(row['Question'], row['AnswerTemplate'], row['AnswExamples'], row['UnknownExample'])

            # Ajouter la réponse à la nouvelle colonne 'reponse'
            df.at[index, 'Reponse'] = response[0]
            df.at[index, 'RetrievedContext'] = response[1]

            # Export multiretrieved_docs to a txt file
            with open(f"retrieved_context_{index}_{rowID}.txt", "w",  encoding='utf-8') as file:
                file.write(str(response[1]))

            # Export multiresponses to a txt file with UTF-8 encoding
            with open(f"reponses_{index}_{rowID}.txt", "w", encoding='utf-8') as file:
                file.write(str(response[0]))
    
    # Enregistrer le DataFrame modifié dans un nouveau fichier Excel
    return df

# Create blank df
df_output6 = add_responses_to_excel(df_comms)
df_output6.info()


Processing Excel rows: 0it [00:00, ?it/s]

"Vous êtes un expert en écologie et environnement de l'entreprise Hermès. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année 2023 en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez ("resilience": NaN).
    
    Context: Here is the context

    Question: E9. L'organisation a-t-elle agi pour soutenir l'adaptation et la résilience au action climatique ? La compagnie a adopté des mesures pour accroître la résilience de l'ensemble de l'organisation face au changement climatique?
    
Format de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \n\nDonnees_extraites = ("resilience": boolean)\n\nAnswer_context=[citations littérales de la source que vous avez utilisées pour répondre, citation2, citation3]

Exemple de bonne réponse: Explication



OllamaEmbeddings: 100%|██████████| 1/1 [01:00<00:00, 60.13s/it]


Explication:

LVMH (Moët Hennessy Louis Vuitton) a pris des mesures pour accroître la résilience de l'ensemble de l'organisation face au changement climatique. Le groupe a mis en place un processus de gestion des risques climatiques qui identifie, évalue et répond aux risques et opportunités liés au climat (DEU: Gestion des risques p. 54). Les Maisons de mode et maroquinerie ont également adopté des pratiques durables telles que la sobriété, l'éternité et la renaissance pour réduire les impacts environnementaux (§2.1.4).

LVMH a également mis en place un processus de validation strict des investissements publicitaires pour minimiser les effets indésirables du climat (§2.3). De plus, le groupe a rejoint l'organisation professionnelle ZDHC (Zero Discharge of Hazardous Chemicals) pour promouvoir les meilleures pratiques en termes d'utilisation de substances dangereuses et de qualité des rejets d'eau usée sur les sites de fabrication (§2.2.4).

Enfin, LVMH a publié des indicateurs pour éva

"Vous êtes un expert en écologie et environnement de l'entreprise Hermès. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année 2023 en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez ("resilience": NaN).
    
    Context: Here is the context

    Question: E9. L'organisation a-t-elle agi pour soutenir l'adaptation et la résilience au action climatique ? Avez-vous adopté des mesures pour accroître la résilience de nos chaînes d'approvisionnement?
    
Format de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citant la page source) \n\nDonnees_extraites = ("resilience": boolean)\n\nAnswer_context=[citations littérales de la source que vous avez utilisées pour répondre, citation2, citation3]

Exemple de bonne réponse: Explication = 'L'entreprise X a exploité pl

...

"Vous êtes un expert en écologie et environnement de l'entreprise Hermès. Votre travail est de répondre à la question donnée ci-après sur la base du contexte donné ci-après datant de l'année 2023 en mentionnant la page du document source, et en respectant le "Format de réponse attendu" donné. Vous répondrez toujours en français. Si vous ne pouvez pas répondre à la question, répondez ('engagementLabourRightsStakeholders': NaN).
    
    Context: Here is the context

    Question: L2. Au cours de la période de reporting, l'entreprise s'est-elle engagée auprès des parties prenantes affectées ou de leurs représentants légitimes en ce qui concerne les questions suivantes relatives aux droits du travail ? Pour l'utilisateur, l'option « Conditions de travail (salaires, heures de travail) » ne serait visible que si elle est sélectionnée dans HR1. Conditions de travail (salaires, heures de travail)
    
Format de réponse attendu: Explication = (Votre réponse en trois paragraphs maximum en citan



OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.10s/it]


Explication: According to the Hermès company's environmental and ecological expertise, there is no specific engagement with stakeholders regarding labor rights during the reporting period. The report does not mention any initiatives or actions taken by the company to address labor rights concerns.

Données_extraites = ('engagementLabourRightsStakeholders': "Aucun engagement sur ce sujet")

Answer_context = [LVMH has remunerated 1,750 million euros in 2020 and 800 million euros and 400 million pounds sterling in 2017. Swaps covering associated with these debt securities have been settled concurrently with the redemptions.]


Processing multiple questions: 100%|██████████| 6/6 [16:10<00:00, 161.83s/it]
Processing Excel rows: 20it [3:28:40, 626.04s/it]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  20 non-null     object
 1   Question            20 non-null     object
 2   RSEframe            20 non-null     object
 3   FrameNet            20 non-null     object
 4   Definition          20 non-null     object
 5   CoreFE              20 non-null     object
 6   NonCoreFE           20 non-null     object
 7   MultiQuestion       20 non-null     bool  
 8   QuestionsMultiples  10 non-null     object
 9   AnswerTemplate      20 non-null     object
 10  AnswExamples        20 non-null     object
 11  Relevantinfo        8 non-null      object
 12  UnknownExample      20 non-null     object
 13  Example_1_Kering    3 non-null      object
 14  Example_2_Kering    3 non-null      object
 15  Example_3_Kering    3 non-null      object
 16  Commentaires        3 non-nu

In [14]:
# Export df_output3 to an excel file
df_output6.to_excel("df_comms_output6LVMH.xlsx", index=False)

# Zero-shot Frame elements extraction

In [17]:
# Lisez le fichier Excel avec les réponses
#df_output4 = pd.read_excel("df_comms_output4.xlsx")

# Afficher le df.info
#df_output4.info()

# Output folder to be used as input of answers from the RAG model
output_folder = "LVMH"

In [ ]:
df_output4

In [111]:
!pip install sentence-splitter

You should consider upgrading via the 'd:\unistra\unistra\recherche\ragbot\env\scripts\python.exe -m pip install --upgrade pip' command.


In [15]:
# Testing reading literal eval the Docs in txt file
import ast
import re

# Define the Document class
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# Function to split sentences using Recursive Character Text Splitter
def split_sentences(docs):
    # Split the content of all documents in docs into sentences
    splitted_sentences = []
    for doc in docs:

         # Replace \xa0 with a space
        doc.page_content = doc.page_content.replace('\xa0', ' ')
        
        # Replace double whitespaces by single whitespace
        doc.page_content = re.sub(' +', ' ', doc.page_content.strip())

        # Split the content into sentences using the regex splitter
        #sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)(?=\s[A-Z])|\n\n', doc.page_content)
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)(?=\s[A-Z]|\s\d)|\n\n', doc.page_content)
        
        # Replace break lines by whitespace
        sentences = [sentence.replace("\n", " ") for sentence in sentences]

        # Remove spaces at the beginning of the sentences
        sentences = [sentence.strip() for sentence in sentences]

        # Replace double whitespaces by single whitespace
        sentences = [re.sub(' +', ' ', sentence) for sentence in sentences]

        # Remove spaces at the beginning of the sentences
        sentences = [sentence.strip() for sentence in sentences]
        
        # Add a final period to the sentences that do not end with a period
        sentences = [sentence.strip() + "." if not sentence.endswith(".") else sentence for sentence in sentences]

        # Append the sentences to the list
        splitted_sentences.extend(sentences)
    return splitted_sentences


# Function to read the content of a txt file and return the content as a list of Document objects
def read_docs_from_txt_file(file_path):
    # Read the content of the file
    with open(file_path, "r", encoding='utf-8') as file:
        # Read the content of the file
        content = file.read()
        # Convert the string to a list of Document objects
        docs = eval(content)
        return docs
    
# Test the function with a txt file
docs = read_docs_from_txt_file("LVMH/retrieved_context_1_E10.txt")

# Print the first three sentences of splited docs
test_sentences = split_sentences(docs)

# Store the length of all sentences in a set, then print
sentence_lengths = {len(sentence) for sentence in test_sentences}
print(test_sentences[0])

print(test_sentences)



RAPPORT DE GESTION DU C ONSEIL D’ADMINISTRATION : LE GROUPE Environnement et durabilité Document d’enregistrement universel 2023 1015.
['RAPPORT DE GESTION DU C ONSEIL D’ADMINISTRATION : LE GROUPE Environnement et durabilité Document d’enregistrement universel 2023 1015.', 'LIFE 360 – CLIMAT La lutte contre le changement climatique est un axe majeur de la politique environnementale de LVMH.', 'Le Groupe a souvent joué un rôle pionnier dans ce domaine.', 'Il a notamment participé au début des années 2000 à l’expérimentation de la méthode qui allait devenir le Bilan Carbone.', 'Il fut aussi la première entreprise, dans l’univers du luxe, à mettre en place en 2015 un fonds carbone interne.', '5.1 Description de la politique Climat Sur la base du calcul complet de l’empreinte carbone globale, mise à jour annuellement par un cabinet externe, LVMH a défini une trajectoire carbone alignée sur l’Accord de Paris.', 'Celle‑ci a été validée en décembre 2021 par le tiers de référence international

In [121]:
df_output4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  10 non-null     object
 1   Question            10 non-null     object
 2   RSEframe            10 non-null     object
 3   FrameNet            10 non-null     object
 4   Definition          10 non-null     object
 5   CoreFE              10 non-null     object
 6   NonCoreFE           10 non-null     object
 7   MultiQuestion       10 non-null     bool  
 8   QuestionsMultiples  7 non-null      object
 9   AnswerTemplate      10 non-null     object
 10  AnswExamples        10 non-null     object
 11  Relevantinfo        7 non-null      object
 12  UnknownExample      10 non-null     object
 13  Example_1_Kering    3 non-null      object
 14  Example_2_Kering    3 non-null      object
 15  Example_3_Kering    3 non-null      object
 16  Commentaires        3 non-nul

In [24]:
# Create an empty dataframe with columns "GRIid", "Frame", "Sentence", "FrameElements", "TriggerWord"
df_frames = pd.DataFrame(columns=["GRIid", "Frame", "Sentence", "Question", "SecondQuestion", "FrameElements", "TriggerWord", "FrameDefinition"])

# Read files in the output folder where their path is built as f"{output_folder}/retrieved_context_{index}_{rowID}_{q_complement_index}.txt"
import os
import pandas as pd

# Iterate over each row in df_output4
for index, row in df_output6.iterrows():
    GRIid = row['ID']
    frame = row['FrameNet']
    frame_definition = row['Definition']
    
    # frame elements merging CoreFE and NonCoreFE after reading their values as literal lists
    coreFEs = ast.literal_eval(row['CoreFE'])
    #print(coreFEs)
    #print(type(coreFEs))
    nonCoreFEs = ast.literal_eval(row['NonCoreFE'])
    #print(nonCoreFEs)
    #print(type(nonCoreFEs))
    frame_elements = coreFEs + nonCoreFEs
    
    # Process files with multiple questions
    if row['MultiQuestion'] == 1:
        multiquestions = ast.literal_eval(row['QuestionsMultiples'])
        for q_complement_index in range(1, len(multiquestions)+1):
            file_path = f"{output_folder}/retrieved_context_{index}_{row['ID']}_{q_complement_index}.txt"
            docs_retrieved = "" 
            sentences = []
            if os.path.exists(file_path):
                
                # Read the docs contained in the file
                docs_retrieved = read_docs_from_txt_file(file_path)

                # Split docs into sentences
                sentences = split_sentences(docs_retrieved)
                
                # Store each sentence in the new dataframe df_frames
                for sentence in sentences:
                    new_row = {"GRIid": GRIid, "Frame": frame,
                               "Sentence": sentence, "Question": row['Question'],
                               "SecondQuestion": multiquestions[q_complement_index-1],
                               "FrameElements": str(frame_elements), "TriggerWord": "",
                               "FrameDefinition": frame_definition}
                    # Append the new row to the df_frames
                    df_frames = pd.concat([df_frames, pd.DataFrame([new_row])], ignore_index=True)
                    
            else:
                print(f"File not found: {file_path}")
    else:

        file_path = f"{output_folder}/retrieved_context_{index}_{row['ID']}.txt"
        
        # Check if the file exists
        if os.path.exists(file_path):
            # Read the docs contained in the file
                docs_retrieved = read_docs_from_txt_file(file_path)

                # Split docs into sentences
                sentences = split_sentences(docs_retrieved)
                
                # Store each sentence in the new dataframe df_frames
                for sentence in sentences:
                    new_row = {"GRIid": GRIid, "Frame": frame, "Sentence": sentence,
                               "Question": row['Question'], "SecondQuestion": "",
                               "FrameElements": str(frame_elements), "TriggerWord": "",
                               "FrameDefinition": frame_definition}
                    # Concat the new row to the df_frames
                    df_frames = pd.concat([df_frames, pd.DataFrame([new_row])], ignore_index=True)

        else:
            print(f"File not found: {file_path}")


# Display the first 5 rows of df_frames
df_frames.tail()

# Drop sentences with less than 200 characters and reset the index
# Print the number of rows of the dataframe before and after dropping
print(df_frames.shape[0])
df_frames = df_frames[df_frames['Sentence'].str.len() > 200]
df_frames = df_frames.reset_index(drop=True)
print(df_frames.shape[0])

2919
1410


In [25]:
# new df
df_frames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1410 entries, 0 to 1409
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   GRIid            1410 non-null   object
 1   Frame            1410 non-null   object
 2   Sentence         1410 non-null   object
 3   Question         1410 non-null   object
 4   SecondQuestion   1410 non-null   object
 5   FrameElements    1410 non-null   object
 6   TriggerWord      1410 non-null   object
 7   FrameDefinition  1410 non-null   object
dtypes: object(8)
memory usage: 88.2+ KB


In [29]:
# Drop rows where Frame = "['placeholder']"
# Print the number of rows of the dataframe before and after dropping
print(df_frames.shape[0])
df_frames = df_frames[df_frames['Frame'] != "['placeholder']"]
df_frames = df_frames.reset_index(drop=True)
print(df_frames.shape[0])

# Keep only the first 10 rows of the dataframe per GRIid and Second Question
# Print the number of rows of the dataframe before and after dropping
print(df_frames.shape[0])
df_frames = df_frames.groupby(['GRIid', 'SecondQuestion']).head(10)
df_frames = df_frames.reset_index(drop=True)
print(df_frames.shape[0])


1287
1287
1287
639


In [30]:
# Export the dataframe to an excel file
df_frames.to_excel("df_frames_sentLVMH_639.xlsx", index=False)

In [ ]:
# Find duplicates in Sentence column
duplicates = df_frames[df_frames.duplicated(subset=["Sentence"], keep=False)]
duplicates

In [27]:
# Create a new Ollama chain to extract the frame elements and trigger words from each sentence in df_frames
def model_frames(sent, frame_description, frame_name, frame_elements):
    

    template_base_french = f""""
    Vous êtes un expert en linguistique, rôles sémantiques et sémantique de cadres (cadres = frames). Un élément de cadre (Frame element) est un rôle sémantique, par example: Agent, Récipient, Instrument, Lieu, Cause. Votre travail comprend 4 tâches sur la phrase donnée ci-après (Sentence): 1, extraire les éléments du cadre Frame_name (description = Frame_definition) seulement parmi les catégories d'éléments données dans Frame_elements_categories; 2, indiquez la position de début (start_loc = starting index in the Sentence string) et de fin (end_loc = ending index in the Sentence string) de chaque Frame_element dans le string de la phrase; 3, extrayez le mot déclencheur (a noun or non-auxiliary verb in the sentence, simple or compound that is linked to the Frame_name) en fonction de la description du cadre donnée. Vous répondrez toujours en français. Si aucun élément de cadre sémantique n'est présent dans la phrase, répondez "Je ne sais pas".

    """ + "Sentence: {sentence}" + f"""
    Frame_name: {frame_name}
    Frame_definition: {frame_description}
    Frame_elements_categories: {frame_elements}

    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: FrameElements = ['FrameElementAnnotation(string=element_name, name=element_category, start_loc = int64, end_loc= int64)', 'FrameElementAnnotation(string=element_name, name=element_category, start_loc = int64, end_loc= int64)'] \\n\\nTriggerWord = (mot déclencheur)"
    
    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(frame_description=frame_description, sentence=sent)
    print(prompt.format(frame_description=frame_description, sentence=sent))
    
    chain = (
    {
        "frame_description": itemgetter("frame_description"),
        "sentence": itemgetter("sentence"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"sentence": sent, "frame_description": frame_description})
    print(result)
    return result

In [31]:
# First 2 rows of df_frames
test_df = df_frames.head(380)
test_df

# Function to apply the model_frames function to each row in the dataframe df_frames
def extract_frame_elements(df):
    # Create a copy of the input dataframe
    df_results = df.copy()
    
    # Iterate over each row in the dataframe
    for index, row in df.iterrows():
        # Call the model_frames function with the values in the row
        result = model_frames(row['Sentence'], row['FrameDefinition'], row['Frame'], row['FrameElements'])
        
        # Store the result in the corresponding Reponse column
        df_results.at[index, 'Reponse'] = str(result)
    
    return df_results

In [ ]:
# Apply the extract_frame_elements function to the test_df
df_results = extract_frame_elements(df_frames)
df_results

In [164]:
# Apply the extract_frame_elements function to the test_df
df_results = extract_frame_elements(df_frames)
df_results

"
    Vous êtes un expert en linguistique, rôles sémantiques et sémantique de cadres (cadre sémantique = semantic frame, a kind of event). Un élément de cadre (Frame element) est un rôle sémantique, par example: Agent, Récipient, Instrument, Lieu, Cause. Votre travail comprend 4 tâches à faire sur la phrase "dans ses opérations, exposition des sites aux aléas climatiques). Concernant sa chaîne amont, Kering travaille notamment sur les axes suivants afin de sécuriser ses approvisionnements à long terme et soutenir les solutions fondées sur la nature, en particulier le développement de l’agriculture régénératrice : •sourcing : mise en œuvre des Kering Standards auprès des fournisseurs en visant 100 % de matières premières alignées avec les Standards en 2025.": 1, extraire les éléments de l'action "Supporting" (description = "A Supporter assists to strengthen the Supported by lending itself in material aid (or abstract assistance in the case of metaphorical uses).") qui sont présents dans

,GRIid,Frame,Sentence,Question,SecondQuestion,FrameElements,TriggerWord,FrameDefinition,Reponse
0,E9,Supporting,"dans ses opérations, exposition des sites aux aléas climatiques). Concernant sa chaîne amont, Kering travaille notamment sur les axes suivants afin de sécuriser ses approvisionnements à long terme et soutenir les solutions fondées sur la nature, en particulier le développement de l’agriculture régénératrice : •sourcing : mise en œuvre des Kering Standards auprès des fournisseurs en visant 100 % de matières premières alignées avec les Standards en 2025.",E9. L'organisation a-t-elle agi pour soutenir l'adaptation et la résilience au action climatique ?,La compagnie a adopté des mesures pour accroître la résilience de l'ensemble de l'organisation face au changement climatique?,"['Supported', 'Supporter', 'Purpose', 'Means', 'Manner', 'Time', 'Instrument']",,A Supporter assists to strengthen the Supported by lending itself in material aid (or abstract assistance in the case of metaphorical uses).,"Après analyse de la phrase, voici les résultats :\n\nFrameElements = ['Supporter', 'Purpose', 'Means']\n['Supported', 51, 58]\n['Supporter', 34, 41]\n['Purpose', 44, 53]\n['Means', 63, 70]\n\nTriggerWord = Soutenir\n\nNote : Les éléments de cadre sémantiques identifiés sont 'Supporter' (Agent), 'Supported' (Theme) et 'Purpose' (Goal). L'élément de cadre sémantique 'Means' est également présent mais il s'agit d'un sous-élément du cadre sémantique 'Supporting'. Le mot déclencheur est le verbe 'soutenir', qui correspond à la description du cadre sémantique 'Supporting'."
1,E9,Supporting,"Ces Standards listent notamment les certifications tierces recommandées dans les chaînes d’approvisionnement du Groupe qui garantissent des pratiques agricoles à faible impact et plus forte résilience, telles que l'agriculture biologique ; •support direct à des programmes de sourcing pour certaines matières premières clés : Kering est par exemple partenaire du Savory Institute pour promouvoir l’approvisionnement en matières premières venant de système de pâturage régénératifs, en particulier pour le cuir bovin ; •Kering Regenerative Fund for Nature : ce Fonds a pour objectif de soutenir la transition vers une agriculture régénératrice auprès de producteurs de matières premières clés pour le Groupe afin d’atteindre en 2025 la transition d’un million d’hectares.",E9. L'organisation a-t-elle agi pour soutenir l'adaptation et la résilience au action climatique ?,La compagnie a adopté des mesures pour accroître la résilience de l'ensemble de l'organisation face au changement climatique?,"['Supported', 'Supporter', 'Purpose', 'Means', 'Manner', 'Time', 'Instrument']",,A Supporter assists to strengthen the Supported by lending itself in material aid (or abstract assistance in the case of metaphorical uses).,"Je vais répondre à vos questions.\n\n**1. Extraire les éléments de l'action ""Supporting""**\n\nLes éléments de l'action ""Supporting"" présents dans la phrase sont :\n\n* 'Supported' : les producteurs de matières premières clés\n* 'Supporter' : Kering, en tant que partenaire du Savory Institute\n* 'Purpose' : promouvoir l’approvisionnement en matières premières venant de système de pâturage régénératifs\n\nFrameElements = ['Supported', 'Supporter', 'Purpose'], 'Means'\n\n**2. Position des éléments**\n\nVoici les positions des éléments :\n\n* 'Supported' : start_index=59, end_index=71\n* 'Supporter' : start_index=64, end_index=75\n* 'Purpose' : start_index=76, end_index=93\n\nFrameElements = ['Supported (59-71)', 'Supporter (64-75)', 'Purpose (76-93)']\n\n**3. Mot déclencheur**\n\nLe mot déclencheur est ""support"".\n\nTriggerWord = support"
2,E9,Supporting,"7 projets sont actuellement soutenus sur les matières premières suivantes : cuir, coton, laine et cachemire (voir section 5.3.2). Ces projets d’agriculture régénératrice soutiennent la résilience climatique.",E9. L'organisation a-t-elle agi pour soutenir l'adaptation et la résilience au action c

In [32]:
# Create a new Ollama chain to extract the frame elements and trigger words from each sentence in df_frames
def model_frames(sent, frame_description, frame_name, frame_elements):
    

    template_base_french = """"
    Vous êtes un expert en linguistique, rôles sémantiques et sémantique de cadres (cadre sémantique = semantic frame, a kind of event). Un élément de cadre (Frame element) est un rôle sémantique, par example: Agent, Récipient, Instrument, Lieu, Cause. Votre travail comprend 3 tâches à faire sur la phrase "{sentence}": 1, extraire les """ + f"""éléments de l'action "{frame_name}" (description = "{frame_description}") qui sont présents dans la phrase seulement parmi les catégories d'éléments suivantes "{frame_elements}"; 2, indiquez la position de début (start_index = starting index in the Sentence string) et de fin (end_index = ending index in the Sentence string) de chaque élément comme si c'était une entité nommée spaCy (['Lieu', 21, 33]) ; 3, extrayez le mot déclencheur (a noun or non-auxiliary verb in the sentence but never a preposition nor a grammar word, it can be simple or compound and it triggers the action related to the Frame_name "{frame_name}") en fonction de la description du cadre sémantique. Vous répondrez toujours en français. Si aucun élément de cadre sémantique n'est présent dans la phrase, répondez "Je ne sais pas".
    """
    template_fr = f"{template_base_french}\nFormat de réponse attendu: Task1_FrameElements = ['element_category':'cited element from sentence', 'element2_category':'second element']. Task2_Indexes = [[21,25], [35, 74]]. Task3_TriggerWord = 'cited trigger word'"
    
    prompt = PromptTemplate.from_template(template_fr)
    prompt.format(frame_description=frame_description, sentence=sent)
    print(prompt.format(frame_description=frame_description, sentence=sent))
    
    chain = (
    {
        "frame_description": itemgetter("frame_description"),
        "sentence": itemgetter("sentence"),
    }
    | prompt
    | model
    | parser
)

    result = chain.invoke({"sentence": sent, "frame_description": frame_description})
    print(result)
    return result


results3 = extract_frame_elements(test_df)
results3
# Export df_results to exce
results3.to_excel("df_frame_resultsLVMH.xlsx", index=False)

"
    Vous êtes un expert en linguistique, rôles sémantiques et sémantique de cadres (cadre sémantique = semantic frame, a kind of event). Un élément de cadre (Frame element) est un rôle sémantique, par example: Agent, Récipient, Instrument, Lieu, Cause. Votre travail comprend 3 tâches à faire sur la phrase "RAPPORT DE GESTION DU C ONSEIL D’ADMINISTRATION : LE GROUPE Environnement et durabilité Document d’enregistrement universel 2023 771.1.2 Identification des risques En 2023, LVMH a initié l’analyse en double matérialité des impacts, risques et opportunités Climat pour le Groupe, de manière à affiner l’identification des principaux enjeux environnementaux : • pour l’impact climat, en 2023, LVMH a conduit une étude permettant d’identifier les risques climatiques principaux sur sa chaîne de valeur.": 1, extraire les éléments de l'action "Supporting" (description = "A Supporter assists to strengthen the Supported by lending itself in material aid (or abstract assistance in the case of m

In [168]:
# Export df_results to exce
results2.to_excel("df_frame_results380.xlsx", index=False)